In [8]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import cx_Oracle
import getpass
import requests

#######Database connection###########
def con_database():
    #dsn_tns = cx_Oracle.makedsn('bandb-at1.test.avc.edu', '1521', service_name='avcprod.avc.edu') # if needed, place an 'r' before any parameter in order to address special characters such as '\'.
    dsn_tns = cx_Oracle.makedsn('bandb-at1.test.avc.edu', '1521', 'avcprep')
    while True:
        try:
            _connection = cx_Oracle.connect(user=getpass.getpass("Username: "), password=getpass.getpass("Password: "),
                                           dsn=dsn_tns, #'AVCPROD', #dsn_tns, #'AVCPREP',
                                           encoding="UTF-8")
            #print(connection)
            return _connection
        except Exception as e:
            print(e)

def close_db_cur():
    cur.close()

def close_db_connection():
    connection.close()

def exit_db():
    close_db_cur()
    close_db_connection()
####### END Database connection###########

In [9]:
connection = con_database()
cur = connection.cursor()
print(connection.version)

19.10.0.0.0


In [22]:
def define_table(table):
    _query = f"""select * from {table}"""
    for column in cur.description:
        print(column)
    #return app_data

define_table('FORAQUS')

('FORAQUS_QUEUE_ID', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 4, 16, None, None, 0)
('FORAQUS_USER_ID_APPR', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 30, 120, None, None, 0)
('FORAQUS_QUEUE_LEVEL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0)
('FORAQUS_QUEUE_LIMIT', <cx_Oracle.DbType DB_TYPE_NUMBER>, 21, None, 17, 2, 0)
('FORAQUS_EFF_DATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 0)
('FORAQUS_NCHG_DATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1)
('FORAQUS_TERM_DATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1)
('FORAQUS_ACTIVITY_DATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 0)
('FORAQUS_USER_ID', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 30, 120, None, None, 0)
('FORAQUS_SURROGATE_ID', <cx_Oracle.DbType DB_TYPE_NUMBER>, 20, None, 19, 0, 0)
('FORAQUS_VERSION', <cx_Oracle.DbType DB_TYPE_NUMBER>, 20, None, 19, 0, 0)
('FORAQUS_DATA_ORIGIN', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 30, 120, None, None, 1)
('FORAQUS_VPDI_CODE', <cx_Oracle.DbTyp

In [57]:
def db_query(cur,_select,_from,_where,_order_by,_group_by,_max_records_returned):
    _select = f"""select  {_select}"""
    _from = f"""from {_from}"""
    _where = (f"""where {_where}""" if _where else '')
    _order_by = (f"""order by {_order_by}""" if _order_by else '')
    _group_by = (f"""group by {_group_by}"""  if _group_by else '')
    _max_records_returned = (f"""fetch next {_max_records_returned} rows only""" if _group_by else '')
    _query = f"""{_select} {_from} {_where} {_order_by} {_group_by} {_max_records_returned}"""

    print(_query)
    cur.execute(_query)
    app_data = cur.fetchall()



    return app_data


In [58]:
select = f"""FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,
FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE"""
from_ = f"""FORAQUS"""
where = f""""""
order_by = f"""FORAQUS_QUEUE_ID"""
group_by = f""""""
max_records_returned = '' #10 #f"""10"""
x = db_query(cur,select,from_,where,order_by,group_by,max_records_returned)
for y in x:
    print(y)

select  FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,
FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE from FORAQUS  order by FORAQUS_QUEUE_ID  
('000A', 'AMUSIAL1', 10, 0.0, datetime.datetime(1999, 7, 1, 0, 0), None, None)
('000A', 'DMCINTOSH1', 10, 0.0, datetime.datetime(1999, 7, 1, 0, 0), datetime.datetime(2019, 7, 22, 13, 49, 4), None)
('000A', 'EOBRIEN3', 10, 0.0, datetime.datetime(2018, 6, 20, 14, 35, 56), datetime.datetime(2019, 7, 22, 13, 48, 34), None)
('000A', 'DMCINTOSH1', 10, 0.0, datetime.datetime(2018, 6, 20, 14, 35, 56), datetime.datetime(2019, 7, 22, 13, 51, 43), None)
('000A', 'JGADDIS', 10, 0.0, datetime.datetime(2017, 11, 22, 10, 30, 38), datetime.datetime(2018, 4, 16, 16, 23, 48), None)
('000A', 'JGADDIS', 10, 0.0, datetime.datetime(1999, 7, 1, 0, 0), datetime.datetime(2017, 11, 22, 10, 30, 38), None)
('000A', 'JGADDIS', 10, 0.0, datetime.datetime(2018, 4, 16, 16, 23, 48), None, datetime.datetime(2018, 4, 16, 0, 0))
('000A', 'EOB

In [ ]:
_select = f"""select FORAQUS_QUEUE_ID,FORAQUS_USER_ID_APPR,FORAQUS_QUEUE_LEVEL,FORAQUS_QUEUE_LIMIT,FORAQUS_EFF_DATE,
FORAQUS_NCHG_DATE,FORAQUS_TERM_DATE"""
_from = f"""from FORAQUS"""
_where = f""""""
_order_by = f""""""
x = db_query(cur,_select,_from,_where,_order_by)
for y in x:
    print(y)